#  Snowflake Cortex AI SQL: Extracting Insights from Multimodal Customer Service Data

## Build Complex SQL Queries with AISQL

In [ ]:
INSERT INTO transcription_results (
    stage_location,
    file_name,
    timestamp_granularity,
    audio_duration,
    segments,
    raw_response,
    translated_text,
    sentiment_label,
    call_category,
    call_summary,
    transcription_completed_at
)
WITH transcriptions AS (
    SELECT 
        file_name,
        AI_TRANSCRIBE(
            TO_FILE('@MULTIMODAL_CUSTOMER_SERVICE.DATA.Customer_Calls/' || file_name),
            OBJECT_CONSTRUCT('timestamp_granularity', 'speaker')
        ) AS trans_result
    FROM data.audio_file_list
),
with_transcripts AS (
    SELECT
        file_name,
        trans_result,
        ARRAY_TO_STRING(
            ARRAY_AGG(seg.value:text::VARCHAR) WITHIN GROUP (ORDER BY seg.index),
            ' '
        ) AS full_transcript
    FROM transcriptions,
         TABLE(FLATTEN(input => trans_result:segments)) seg
    GROUP BY file_name, trans_result
),
with_translation AS (
    SELECT
        file_name,
        trans_result,
        full_transcript,
        AI_TRANSLATE(full_transcript,'' ,'en') AS translated_transcript
    FROM with_transcripts
)
SELECT
    '@MULTIMODAL_CUSTOMER_SERVICE.DATA.Customer_Calls/' || file_name AS stage_location,
    file_name,
    'speaker' AS timestamp_granularity,
    trans_result:audio_duration::FLOAT AS audio_duration,
    trans_result:segments AS segments,
    trans_result AS raw_response,
    translated_transcript AS translated_text,
    AI_SENTIMENT(translated_transcript):categories[0]:sentiment::VARCHAR AS sentiment_label,
    AI_CLASSIFY(
        translated_transcript,
        [
            OBJECT_CONSTRUCT('label', 'Fraud & Security Issues', 
                           'description', 'Unauthorized transactions, identity theft, account freezes, fraudulent charges'),
            OBJECT_CONSTRUCT('label', 'Technical & System Errors',
                           'description', 'Auto-pay failures, system glitches, login problems, display issues, platform malfunctions'),
            OBJECT_CONSTRUCT('label', 'Payment & Transaction Problems',
                           'description', 'Duplicate charges, failed payments, processing errors, fee disputes, rate increases'),
            OBJECT_CONSTRUCT('label', 'Account Changes & Modifications',
                           'description', 'Investment reallocation, policy modifications, coverage changes, fund transfers, limit adjustments'),
            OBJECT_CONSTRUCT('label', 'General Inquiries & Information Requests',
                           'description', 'Status checks, documentation requests, simple questions, coverage verification, timeline questions')
        ],
        OBJECT_CONSTRUCT('task_description', 'Classify customer service calls into issue categories based on the main problem discussed')
    ):labels[0]::VARCHAR AS call_category,
    AI_COMPLETE(
        'claude-sonnet-4-5',
        CONCAT('Summarize this call in 50 words: ', translated_transcript)
    ) AS call_summary,
    CURRENT_TIMESTAMP() AS transcription_completed_at
FROM with_translation;

## AI_TRANSCRIBE

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_transcriptions AS
SELECT 
    file_name,
    AI_TRANSCRIBE(
        TO_FILE('@MULTIMODAL_CUSTOMER_SERVICE.DATA.Customer_Calls/' || file_name),
        OBJECT_CONSTRUCT('timestamp_granularity', 'speaker')
    ) AS trans_result
FROM data.audio_file_list;

SELECT * FROM temp_transcriptions LIMIT 5;

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_with_transcripts AS
SELECT
    file_name,
    trans_result,
    ARRAY_TO_STRING(
        ARRAY_AGG(seg.value:text::VARCHAR) WITHIN GROUP (ORDER BY seg.index),
        ' '
    ) AS full_transcript
FROM temp_transcriptions,
     TABLE(FLATTEN(input => trans_result:segments)) seg
GROUP BY file_name, trans_result;

SELECT * FROM temp_with_transcripts LIMIT 5;

## AI_TRANSLATE

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_with_translation AS
SELECT
    file_name,
    trans_result,
    full_transcript,
    AI_TRANSLATE(full_transcript, '', 'en') AS translated_transcript
FROM temp_with_transcripts;

SELECT * FROM temp_with_translation LIMIT 5;

## AI_SENTIMENT

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_with_sentiment AS
SELECT
    file_name,
    trans_result,
    full_transcript,
    translated_transcript,
    AI_SENTIMENT(translated_transcript):categories[0]:sentiment::VARCHAR AS sentiment_label
FROM temp_with_translation;

SELECT * FROM temp_with_sentiment LIMIT 5;

## AI_CLASSIFY

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_with_classification AS
SELECT
    file_name,
    trans_result,
    full_transcript,
    translated_transcript,
    sentiment_label,
    AI_CLASSIFY(
        translated_transcript,
        [
            OBJECT_CONSTRUCT('label', 'Fraud & Security Issues', 
                           'description', 'Unauthorized transactions, identity theft, account freezes, fraudulent charges'),
            OBJECT_CONSTRUCT('label', 'Technical & System Errors',
                           'description', 'Auto-pay failures, system glitches, login problems, display issues, platform malfunctions'),
            OBJECT_CONSTRUCT('label', 'Payment & Transaction Problems',
                           'description', 'Duplicate charges, failed payments, processing errors, fee disputes, rate increases'),
            OBJECT_CONSTRUCT('label', 'Account Changes & Modifications',
                           'description', 'Investment reallocation, policy modifications, coverage changes, fund transfers, limit adjustments'),
            OBJECT_CONSTRUCT('label', 'General Inquiries & Information Requests',
                           'description', 'Status checks, documentation requests, simple questions, coverage verification, timeline questions')
        ],
        OBJECT_CONSTRUCT('task_description', 'Classify customer service calls into issue categories based on the main problem discussed')
    ):labels[0]::VARCHAR AS call_category
FROM temp_with_sentiment;


SELECT * FROM temp_with_classification LIMIT 5;

## AI_COMPLETE

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_with_summary AS
SELECT
    file_name,
    trans_result,
    full_transcript,
    translated_transcript,
    sentiment_label,
    call_category,
    AI_COMPLETE(
        'claude-sonnet-4-5',
        CONCAT('Summarize this call in 50 words: ', translated_transcript)
    ) AS call_summary
FROM temp_with_classification;

SELECT * FROM temp_with_summary LIMIT 5;

## Combine them all

In [ ]:
INSERT INTO transcription_results (
    stage_location,
    file_name,
    timestamp_granularity,
    audio_duration,
    segments,
    raw_response,
    translated_text,
    sentiment_label,
    call_category,
    call_summary,
    transcription_completed_at
)
SELECT
    '@MULTIMODAL_CUSTOMER_SERVICE.DATA.Customer_Calls/' || file_name AS stage_location,
    file_name,
    'speaker' AS timestamp_granularity,
    trans_result:audio_duration::FLOAT AS audio_duration,
    trans_result:segments AS segments,
    trans_result AS raw_response,
    translated_transcript AS translated_text,
    sentiment_label,
    call_category,
    call_summary,
    CURRENT_TIMESTAMP() AS transcription_completed_at
FROM temp_with_summary;

## CLEAN UP

In [ ]:
DROP TABLE IF EXISTS temp_transcriptions;
DROP TABLE IF EXISTS temp_with_transcripts;
DROP TABLE IF EXISTS temp_with_translation;
DROP TABLE IF EXISTS temp_with_sentiment;
DROP TABLE IF EXISTS temp_with_classification;
DROP TABLE IF EXISTS temp_with_summary;

 ## RESULTS

In [ ]:
SELECT * FROM transcription_results;

## Process PDFS

In [ ]:
CREATE TABLE IF NOT EXISTS parsed_documents_raw AS
SELECT 
    RELATIVE_PATH AS file_name,
    '@COMPANY_DOCUMENTS/' || RELATIVE_PATH AS file_path,
    SIZE AS file_size_bytes,
    LAST_MODIFIED,
    AI_PARSE_DOCUMENT(
        TO_FILE('@COMPANY_DOCUMENTS', RELATIVE_PATH),
        {'mode': 'LAYOUT', 'page_split': TRUE}
    ) AS parsed_result
FROM DIRECTORY(@COMPANY_DOCUMENTS);

Select * from parsed_documents_raw limit 10;

# Process Chat Logs and Support Tickets

In [ ]:
CREATE OR REPLACE TABLE chat_validation_results AS
WITH conversation_prep AS (
    SELECT 
        c.chat_id,
        c.ticket_id,
        c.customer_name,
        c.agent_name,
        c.chat_timestamp,
        c.self_reported_category,
        c.self_reported_sentiment,
        c.resolution_status,
        
        -- Build conversation text from messages array
        ARRAY_TO_STRING(
            ARRAY_AGG(msg.value:message::VARCHAR) 
            WITHIN GROUP (ORDER BY msg.index), 
            ' '
        ) AS full_conversation
        
    FROM chat_logs c,
    LATERAL FLATTEN(input => c.messages) msg
    GROUP BY 
        c.chat_id, c.ticket_id, c.customer_name, c.agent_name,
        c.chat_timestamp, c.self_reported_category, 
        c.self_reported_sentiment, c.resolution_status
),
ai_analysis AS (
    SELECT 
        chat_id,
        ticket_id,
        customer_name,
        agent_name,
        chat_timestamp,
        self_reported_category,
        self_reported_sentiment,
        resolution_status,
        full_conversation,
        
        AI_CLASSIFY(
            full_conversation,
            ARRAY_CONSTRUCT('Technical Support', 'Bug Report', 'Feature Request', 'Billing', 'General Inquiry')
        ) AS ai_classify_object,
        
        AI_SENTIMENT(full_conversation) AS ai_sentiment_object,
        
        AI_EXTRACT(
            text => full_conversation,
            responseFormat => OBJECT_CONSTRUCT(
                'issue_description', 'What is the main issue or problem described?',
                'product_name', 'What product is mentioned?',
                'error_message', 'What error message or code is mentioned?',
                'resolution_provided', 'What solution or resolution was provided?',
                'customer_satisfaction_indicator', 'Does the customer seem satisfied?',
                'urgency_level', 'How urgent is this issue?'
            )
        ) AS extracted_features
        
    FROM conversation_prep
)
SELECT 
    chat_id,
    ticket_id,
    customer_name,
    agent_name,
    chat_timestamp,
    self_reported_category,
    self_reported_sentiment,
    resolution_status,
    full_conversation,
    
    ai_classify_object:labels[0]::VARCHAR AS ai_classified_category,
    
    -- Extract sentiment from the categories array
    -- AI_SENTIMENT returns: {"categories": [{"name": "overall", "sentiment": "negative"}]}
    ai_sentiment_object:categories[0]:sentiment::VARCHAR AS ai_sentiment_raw,
    
    -- Normalize AI sentiment to match your categories (Frustrated = Negative)
    CASE 
        WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'positive' THEN 'Positive'
        WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'neutral' THEN 'Neutral'
        WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR IN ('negative', 'very_negative') THEN 'Negative'
        WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'mixed' THEN 'Mixed'
        ELSE 'Unknown'
    END AS ai_sentiment_normalized,
    
    extracted_features,
    CASE 
        -- Category mismatch
        WHEN self_reported_category != ai_classify_object:labels[0]::VARCHAR 
         AND ai_classify_object:labels[0]::VARCHAR IS NOT NULL THEN TRUE
        
        -- Sentiment mismatch (accounting for Frustrated = Negative)
        WHEN self_reported_sentiment != CASE 
                WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'positive' THEN 'Positive'
                WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'neutral' THEN 'Neutral'
                WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR IN ('negative', 'very_negative') THEN 'Negative'
                ELSE 'Unknown'
            END
         AND NOT (self_reported_sentiment = 'Frustrated' 
                  AND ai_sentiment_object:categories[0]:sentiment::VARCHAR IN ('negative', 'very_negative'))
         AND ai_sentiment_object:categories[0]:sentiment::VARCHAR IS NOT NULL THEN TRUE
        
        -- Mixed sentiment detected
        WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'mixed' THEN TRUE
        
        ELSE FALSE
    END AS is_flagged,
    
    -- Array of flag reasons (can have multiple)
    ARRAY_CONSTRUCT_COMPACT(
        CASE 
            WHEN self_reported_category != ai_classify_object:labels[0]::VARCHAR
             AND ai_classify_object:labels[0]::VARCHAR IS NOT NULL
            THEN 'category_mismatch'
        END,
        CASE 
            WHEN self_reported_sentiment != CASE 
                    WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'positive' THEN 'Positive'
                    WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'neutral' THEN 'Neutral'
                    WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR IN ('negative', 'very_negative') THEN 'Negative'
                    ELSE 'Unknown'
                END
             AND NOT (self_reported_sentiment = 'Frustrated' 
                      AND ai_sentiment_object:categories[0]:sentiment::VARCHAR IN ('negative', 'very_negative'))
             AND ai_sentiment_object:categories[0]:sentiment::VARCHAR IS NOT NULL
            THEN 'sentiment_mismatch'
        END,
        CASE 
            WHEN ai_sentiment_object:categories[0]:sentiment::VARCHAR = 'mixed' 
            THEN 'mixed_sentiment'
        END
    ) AS flag_reasons,
    
    CURRENT_TIMESTAMP() AS validation_timestamp

FROM ai_analysis;

SELECT * FROM chat_validation_results LIMIT 5;

In [ ]:
CREATE OR REPLACE TABLE ticket_chat_alignment AS
WITH ticket_chat_data AS (
    SELECT 
        t.ticket_id,
        t.ticket_number,
        t.customer_name AS ticket_customer_name,
        t.subject AS ticket_subject,
        t.description AS ticket_description,
        t.self_reported_category AS ticket_category,
        t.priority AS ticket_priority,
        t.status AS ticket_status,
        t.product_affected AS ticket_product,
        
        cv.chat_id,
        cv.customer_name AS chat_customer_name,
        cv.self_reported_category AS chat_category,
        cv.self_reported_sentiment AS chat_sentiment,
        c.product_mentioned AS chat_product,
        cv.full_conversation
        
    FROM support_tickets t
    INNER JOIN chat_validation_results cv ON t.ticket_id = cv.ticket_id
    INNER JOIN chat_logs c ON cv.chat_id = c.chat_id
),
ai_responses AS (
    SELECT 
        ticket_id,
        ticket_number,
        chat_id,
        ticket_customer_name,
        chat_customer_name,
        ticket_category,
        chat_category,
        ticket_product,
        chat_product,
        ticket_subject,
        ticket_description,
        full_conversation,
        
        -- Get AI response
        AI_COMPLETE(
            model => 'claude-sonnet-4-5',
            prompt => 'Compare the ticket and chat to determine if they discuss the same issue.\n\n' ||
                'TICKET:\n' ||
                'Subject: ' || ticket_subject || '\n' ||
                'Description: ' || ticket_description || '\n' ||
                'Category: ' || ticket_category || '\n' ||
                'Product: ' || COALESCE(ticket_product, 'Not specified') || '\n\n' ||
                'CHAT:\n' ||
                'Conversation: ' || full_conversation || '\n' ||
                'Category: ' || chat_category || '\n' ||
                'Product: ' || COALESCE(chat_product, 'Not mentioned') || '\n\n' ||
                'Analyze if the ticket and chat are about the SAME core issue. Consider:\n' ||
                '1. Is the main problem/request the same?\n' ||
                '2. Is the product/service the same?\n' ||
                '3. Is the technical issue consistent?\n\n' ||
                'Respond ONLY with valid JSON (no markdown formatting):\n' ||
                '{"alignment":"aligned|misaligned|partial","confidence":"high|medium|low",' ||
                '"reason":"brief explanation","severity":"critical|moderate|minor"}',
            model_parameters => OBJECT_CONSTRUCT('temperature', 0.1)
        ) AS ai_alignment_response_raw,
        
        -- Clean markdown formatting from response
        REGEXP_REPLACE(
            REGEXP_REPLACE(ai_alignment_response_raw, '^```json\\s*', ''),
            '\\s*```$', 
            ''
        ) AS ai_alignment_response
        
    FROM ticket_chat_data
)
SELECT 
    ticket_id,
    ticket_number,
    chat_id,
    ticket_customer_name,
    chat_customer_name,
    ticket_category,
    chat_category,
    ticket_product,
    chat_product,
    ticket_subject,
    ticket_description,
    full_conversation,
    ai_alignment_response,
    
    -- Parse the cleaned AI response
    PARSE_JSON(ai_alignment_response):alignment::VARCHAR AS alignment_status,
    PARSE_JSON(ai_alignment_response):confidence::VARCHAR AS alignment_confidence,
    PARSE_JSON(ai_alignment_response):reason::VARCHAR AS alignment_reason,
    PARSE_JSON(ai_alignment_response):severity::VARCHAR AS misalignment_severity,
    
    -- Simple category comparison flag
    CASE 
        WHEN ticket_category = chat_category THEN FALSE
        ELSE TRUE
    END AS category_mismatch_flag,
    
    -- Product comparison flag
    CASE 
        WHEN ticket_product IS NOT NULL 
         AND chat_product IS NOT NULL
         AND LOWER(ticket_product) != LOWER(chat_product) 
        THEN TRUE
        ELSE FALSE
    END AS product_mismatch_flag,
    
    -- Consolidated flagging structure
    CASE 
        WHEN PARSE_JSON(ai_alignment_response):alignment::VARCHAR = 'misaligned' THEN TRUE
        WHEN PARSE_JSON(ai_alignment_response):alignment::VARCHAR = 'partial' 
         AND PARSE_JSON(ai_alignment_response):severity::VARCHAR IN ('critical', 'moderate') THEN TRUE
        WHEN ticket_category != chat_category THEN TRUE
        WHEN ticket_product IS NOT NULL 
         AND chat_product IS NOT NULL
         AND LOWER(ticket_product) != LOWER(chat_product) THEN TRUE
        ELSE FALSE
    END AS is_flagged,
    
    -- Array of flag reasons
    ARRAY_CONSTRUCT_COMPACT(
        CASE 
            WHEN PARSE_JSON(ai_alignment_response):alignment::VARCHAR = 'misaligned' 
            THEN 'issue_misalignment'
        END,
        CASE 
            WHEN PARSE_JSON(ai_alignment_response):alignment::VARCHAR = 'partial' 
            THEN 'partial_alignment'
        END,
        CASE 
            WHEN ticket_category != chat_category 
            THEN 'category_mismatch'
        END,
        CASE 
            WHEN ticket_product IS NOT NULL 
             AND chat_product IS NOT NULL
             AND LOWER(ticket_product) != LOWER(chat_product)
            THEN 'product_mismatch'
        END
    ) AS flag_reasons,
    
    CURRENT_TIMESTAMP() AS validation_timestamp

FROM ai_responses;

SELECT * FROM ticket_chat_alignment LIMIT 5;